# Voyages API Voyages Like Use Case

A Voyage is defined to be a sequence of Load operations followed by a sequence of Discharges. Users of **Signal Ocean Platform** interface with the concept of a voyage in different levels of detail. For example in the Voyages tab of Vessels Data (https://app.signalocean.com/vessels) users can see historically all the operations of a specific vessel even at jetty level.  
However very often arises the need of conducting an analysis of the voyages for a specific vessel class for a specific time window. This need is accommodated by the  **Voyages Data Dashboard** (https://app.signalocean.com/reportsindex/voyagesdatalive).  

The level of detail provided by the Voyages Data Dashboard has been tailored, having in mind the neccessary information needed to carry out such an analysis without being overwhelmed by the full data provided by Signal Ocean Platform regarding the voyages of the vessels.  

While both ```get_voyages``` and ``get_voyages_flat`` functions of the Signal SDK return the full low level data available, in this example we are going to construct a dataframe that resembles the form of ***Voyages Data Dashboard***

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
Set your subscription key acquired here: https://apis.signalocean.com/profile

In [1]:
signal_ocean_api_key = '' #replace with your subscription key

## Voyages API Voyages Like Use Case

In [2]:
from signal_ocean import Connection
from signal_ocean.voyages import VoyagesAPI
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime, timezone
import datetime as dt

In [4]:
connection = Connection(signal_ocean_api_key)
api = VoyagesAPI(connection)

### Get voyages

In [5]:
vlcc_id = 84
date_from = datetime.now(timezone.utc) - timedelta(days=365)

Given the definition of a voyage, a new one offsets when a loading operation is executed by a ballast vessel. However it is usefull to display information regarding the opening of the vessel. The opening of a vessel is equivalent with its last discharge.

In order to acquire the opening info of a vessel, we need a ``relaxed_date_from`` variable.  
Since the last discharge of a ballast vessel is not part of its current voyage we need this date so as to retrieve voyages prior to the date of interest, and so to get the last discharge of the last voyage of the vessel. We set the timedelta to 365 days in order to achieve a high confidence level, that the last voyage of the vessel is going to be fetched by the api.

In [6]:
relaxed_date_from = date_from - timedelta(days=360)
voyages = api.get_voyages(vessel_class_id=vlcc_id, date_from=relaxed_date_from)

In [7]:
voyages = pd.DataFrame(v.__dict__ for v in voyages)
events = pd.DataFrame(e.__dict__ for voyage_events in voyages['events'].dropna() for e in voyage_events)

In [8]:
voyages_of_interest = voyages[voyages['start_date'] >= pd.to_datetime(date_from)].copy()

In [9]:
voyages.reset_index(inplace=True)

In [10]:
voyage_ids = voyages.set_index(['imo', 'voyage_number'])['id'].to_dict()
previous_voyage_id = {voyage_id: voyage_ids[(imo, voyage_number-1)] 
                  for voyage_id, imo, voyage_number in voyages[['id', 'imo', 'voyage_number']].values 
                  if (imo, voyage_number-1) in voyage_ids}
voyages_of_interest['prev_id'] = voyages.id.map(previous_voyage_id)
voyages.set_index('id',inplace=True)

In [11]:
def get_opening_event(id_of_prev_voy):
    return next((e.__dict__ for e in reversed(voyages.loc[id_of_prev_voy].events or []) if e.purpose=='Discharge'),None)

def get_load_discharge_events(voyage_events):
    load_event = next((e.__dict__ for e in voyage_events or [] if e.purpose=='Load'), None)
    discharge_event = next((e.__dict__ for e in voyage_events or [] if e.purpose=='Load'), None)
    return pd.Series((load_event, discharge_event))
    
voyages_of_interest.loc[voyages_of_interest.prev_id.notna(),'opening_event'] = \
voyages_of_interest.loc[voyages_of_interest.prev_id.notna(),'prev_id'].apply(get_opening_event)

voyages_of_interest[['load_event','discharge_event']] = voyages_of_interest['events'].apply(get_load_discharge_events)

In [12]:
mapping_dict = {'port_name':['open_port','load_port','discharge_port'],
                'area_name_level0':['starting_narrow_area','load_port_narrow_area','discharge_port_narrow_area'], 
                'area_name_level2':['starting_wide_area','load_port_wide_area','discharge_port_wide_area'], 
                'area_name_level1':['starting_area','load_port_area','discharge_port_area'], 
                'sailing_date':['starting_port_sail_date','load_port_sailing_date','discharge_port_sailing_date'], 
                'longitude':['starting_port_longitude','load_port_longitude','discharge_port_longitude'], 
                'latitude':['starting_port_latitude','load_port_latitude','discharge_port_latitude']}

events = {0:'opening_event',1:'load_event',2:'discharge_event'}

In [13]:
for feature,targets in mapping_dict.items():
    for num,target in enumerate(targets):
        voyages_of_interest[target] = voyages_of_interest[events[num]].apply(lambda e: e[feature] if isinstance(e,dict) else None)

In [14]:
def get_last_discharge_port(voyage):
    if not voyage['discharge_event']:
        return voyage['open_port']
    else:
        return next((e.port_name for e in reversed(voyage['events'][:-2] or []) if e.purpose=='Discharge'), None)
    
voyages_of_interest['last_discharge_port'] = voyages_of_interest.apply(get_last_discharge_port, axis=1)

In [15]:
def get_last_3_months_ind(laycan_from):
    if not pd.isnull(laycan_from):
        laycan_from = pd.to_datetime(laycan_from)
        return 1 if ((laycan_from.date()-date.today()).days<4*30) else 0
    else:
        return 0
    
voyages['last_3_months_ind'] = voyages['laycan_from'].apply(get_last_3_months_ind)

In [16]:
def get_sts_load_ind(load_event):
    return next((1 for d in load_event["event_details"] or [] if d.event_detail_type =='StS'), 0)

def get_sts_discharge_ind(discharge_event):
    return next((1 for d in discharge_event["event_details"] or [] if d.event_detail_type =='StS'), 0)


voyages_of_interest.loc[voyages_of_interest.discharge_event.notna(),'sts_discharge_ind'] = \
voyages_of_interest.loc[voyages_of_interest.discharge_event.notna(),'discharge_event'].apply(get_sts_discharge_ind)
voyages_of_interest.loc[voyages_of_interest.discharge_event.notna(),'sts_load_ind'] = \
voyages_of_interest.loc[voyages_of_interest.discharge_event.notna(),'load_event'].apply(get_sts_load_ind)

In [17]:
vessel_status_dict = {
    1:"Voyage", 2:"Breaking", 3:"Domestic Trade", 4:"FPSO", 5:"FPSO Conversion", 
    6:"Inactive", 7:"Storage Vessel", 9:"Conversion"
}
voyages_of_interest['vessel_status'] = voyages_of_interest.vessel_status_id.replace(vessel_status_dict)

In [18]:
wanted_columns = ['imo',
                  'voyage_number',
                  'vessel_type_id',
                  'id',
                  'vessel_name',
                  'vessel_type',
                  'vessel_class',
                  'trade',
                  'commercial_operator',
                  'charterer',
                  'rate',
                  'rate_type',
                  'cargo_type',
                  'cargo_group',
                  'quantity',
                  'laycan_from',
                  'laycan_to',
                  'fixture_status_id',
                  'fixture_status',
                  'fixture_date',
                  'fixture_is_coa',
                  'fixture_is_hold',
                  'prev_id',
                  'opening_event',
                  'load_event',
                  'discharge_event',
                  'open_port',
                  'load_port',
                  'discharge_port',
                  'starting_narrow_area',
                  'load_port_narrow_area',
                  'discharge_port_narrow_area',
                  'starting_wide_area',
                  'load_port_wide_area',
                  'discharge_port_wide_area',
                  'starting_area',
                  'load_port_area',
                  'discharge_port_area',
                  'starting_port_sail_date',
                  'load_port_sailing_date',
                  'discharge_port_sailing_date',
                  'starting_port_longitude',
                  'load_port_longitude',
                  'discharge_port_longitude',
                  'starting_port_latitude',
                  'load_port_latitude',
                  'discharge_port_latitude',
                  'last_discharge_port',
                  'sts_discharge_ind',
                  'sts_load_ind']

voyages_of_interest = voyages_of_interest[wanted_columns]

In [19]:
voyages_of_interest = voyages_of_interest.astype(
                                                {
                                                    'laycan_from':'datetime64[ns]', 
                                                    'laycan_to':'datetime64[ns]',
                                                    'fixture_date':'datetime64[ns]',
                                                    'laycan_to':'datetime64[ns]',
                                                    'fixture_date':'datetime64[ns]',
                                                    'starting_port_sail_date':'datetime64[ns]', 
                                                    'load_port_sailing_date':'datetime64[ns]',
                                                    'discharge_port_sailing_date':'datetime64[ns]'
                                                })

In [20]:
voyages_of_interest.to_excel('voyages_data.xlsx')

In [22]:
voyages.columns

Index(['index', 'imo', 'voyage_number', 'vessel_type_id', 'vessel_class_id',
       'vessel_status_id', 'commercial_operator_id', 'deleted', 'events',
       'vessel_name', 'vessel_type', 'vessel_class', 'trade', 'trade_id',
       'vessel_status', 'commercial_operator', 'start_date', 'end_date',
       'charterer_id', 'charterer', 'rate', 'rate_type', 'ballast_bonus',
       'ballast_bonus_type', 'cargo_type_id', 'cargo_type', 'cargo_group_id',
       'cargo_group', 'cargo_type_source', 'quantity', 'laycan_from',
       'laycan_to', 'fixture_status_id', 'fixture_status', 'fixture_date',
       'fixture_is_coa', 'fixture_is_hold', 'last_3_months_ind'],
      dtype='object')